# Settings

In [4]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rnd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
%matplotlib inline

/Users/mdeblaauw/anaconda3/envs/research-paper/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load data

In [8]:
path = '../../omniglot_images/'
data_subsets = ["train", "val", "test"]

data = {}
categories = {}
info = {}
        
for name in data_subsets:
    file_path = os.path.join(path, name + ".pickle")
    print("loading data from {}".format(file_path))
    with open(file_path,"rb") as f:
        (X,c) = pickle.load(f)
        data[name] = X
        categories[name] = c

loading data from ../../omniglot_images/train.pickle
loading data from ../../omniglot_images/val.pickle
loading data from ../../omniglot_images/test.pickle


In [9]:
def create_train_data(size, s='train'):
    #get train data and shape
    X=data[s]
    n_classes, n_examples, w, h = X.shape
    
    #initialize 2 empty arrays for the input size in a list
    pairs=[np.zeros((size, h, w,1)) for i in range(2)]
    
    #initialize vector for the targets
    targets=np.zeros((size,1))
    
    for x in range(size):
        #randomly sample one class (character)
        category = rnd.choice(n_classes,1,replace=False)
        #randomly sample one example from class (1-20 characters)
        idx_1 = rnd.randint(0, n_examples)
        pairs[0][x,:,:,:] = X[category, idx_1].reshape(w, h, 1)
        #randomly sample again one example from class and add last class with modulo
        # ..to ensure not same class pairs are created
        idx_2 = (idx_1 + rnd.randint(0, n_examples)) % n_examples
        #pick images of different class for 1st half and same class for 2nd half
        if x >= size // 2:
            category_2 = category
            targets[x] = 1
        else: 
        #add a random number to the category modulo n classes to ensure 2nd image has
        # ..different category
            idx_2 = rnd.randint(0, n_examples) 
            category_2 = (category + rnd.randint(1,n_classes)) % n_classes
            targets[x] = 0
        pairs[1][x,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
        
    return pairs, targets

In [10]:
train_set, train_labels = create_train_data(10000)
val_set, val_labels = create_train_data(3000)

# Create graph

In [11]:
def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = rnd.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)

In [12]:
def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=rnd.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)

In [13]:
input_shape = (105, 105, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)
#build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Conv2D(64,(10,10),activation='relu',input_shape=input_shape,
                   kernel_initializer=W_init,kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(7,7),activation='relu',
                   kernel_regularizer=l2(2e-4),kernel_initializer=W_init,bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(256,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(Flatten())
convnet.add(Dense(4096,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init))

#call the convnet Sequential model on each of the input tensors so params will be shared
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
#layer to merge two encoded inputs with the l1 distance between them
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#call this layer on list of two input tensors.
L1_distance = L1_layer([encoded_l, encoded_r])
prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

optimizer = Adam(lr=0.00006, beta_1=0.9, beta_2=0.999)
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.count_params()

38951745

In [14]:
def accuracy(pred, true_val):
    acc_bool = np.equal(np.round_(pred), true_val)
    acc = np.mean(acc_bool.astype(int))
    
    return(acc)

# Training

This is just as fast as the Tensorflow implementation

In [18]:
#Training loop
print("!")
batch_size = 128
total_batch = int(10000/batch_size)
total_batch_val = int(3000/batch_size)
epoch = 10

print("training")
for i in range(epoch):
    batch_x1, batch_x2, batch_y = shuffle(train_set[0],train_set[1], train_labels, n_samples = batch_size)
    train_batch_acc = 0
    for j in range(total_batch):
        loss=siamese_net.train_on_batch([batch_x1, batch_x2],batch_y)
        probs = siamese_net.predict([batch_x1, batch_x2])
        train_batch_acc =+ accuracy(probs, batch_y)
        print('Loss:', loss)
        print('Batch:', j)
    train_acc = train_batch_acc/total_batch
    val_batch_acc = 0
    for validation in range(total_batch_val):
        batch_x1, batch_x2, batch_y = shuffle(val_set[0],val_set[1], val_labels, n_samples = batch_size)
        probs = siamese_net.predict([batch_x1, batch_x2])
        val_batch_acc =+ accuracy(probs, batch_y)
    val_acc = val_batch_acc/total_batch_val
    print('Train accuracy:', train_acc)
    print('Validation accuracy:', val_acc)

!
training
Loss: 4.416229
Batch: 0
Loss: 4.35786
Batch: 1
Loss: 4.287537
Batch: 2
Loss: 4.216594
Batch: 3
Loss: 4.142154
Batch: 4
Loss: 4.067619
Batch: 5
Loss: 3.9931695
Batch: 6
Loss: 3.9210038
Batch: 7
Loss: 3.8525903
Batch: 8
Loss: 3.7894886
Batch: 9
Loss: 3.7329462
Batch: 10
Loss: 3.6829195
Batch: 11
Loss: 3.6387866
Batch: 12
Loss: 3.600032
Batch: 13
Loss: 3.5658648
Batch: 14
Loss: 3.53525
Batch: 15
Loss: 3.5072691
Batch: 16
Loss: 3.4812582
Batch: 17
Loss: 3.4566553
Batch: 18
Loss: 3.4330444
Batch: 19
Loss: 3.4101796
Batch: 20
Loss: 3.3878486
Batch: 21
Loss: 3.3658955
Batch: 22
Loss: 3.3442175
Batch: 23
Loss: 3.3227396
Batch: 24
Loss: 3.3014138
Batch: 25
Loss: 3.2802022
Batch: 26
Loss: 3.2590814
Batch: 27
Loss: 3.2380357
Batch: 28
Loss: 3.217055
Batch: 29
Loss: 3.1961517
Batch: 30
Loss: 3.1752927
Batch: 31
Loss: 3.1544974
Batch: 32
Loss: 3.1337705
Batch: 33
Loss: 3.1130958
Batch: 34
Loss: 3.0924904
Batch: 35
Loss: 3.071966
Batch: 36
Loss: 3.0514975
Batch: 37
Loss: 3.0311148
Batch: 

ValueError: operands could not be broadcast together with shapes (3000,1) (128,1) 